In [1]:
import cv2
from mtcnn.mtcnn import MTCNN
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from imutils import paths
from keras.applications import imagenet_utils
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from keras import backend as K
from keras import applications
from keras.models import Sequential, Model, model_from_json
from keras.layers import *
from keras.optimizers import *
from keras.models import Sequential, Model
from keras import applications
import numpy as np
import random
import os
import pandas as pd
from matplotlib import pyplot as plt

Using TensorFlow backend.


In [2]:
img_rows = 224
img_cols = 224
number_epochs = 20
batch_size = 24
number_chance = 3
thresh_stop = 30
face_train =  pd.read_csv('train.csv').iloc[300:600]
label_train = face_train['label'].values
label_train = np.array(label_train)
label_train.shape

(300,)

In [3]:
image_csv = face_train['image'].values
image_path = list(paths.list_images('train/'))
image_path_array = np.array(image_path)
image_path_append = []
for line in image_path_array:
    name_image = line.split('/')
    name_image = name_image[1]
    image_path_append.append(name_image)
train_images = set(image_path_append)

# Create a list of all the training images with their full path names
train_img = []
for i in image_csv: # img is list of full path names of all images
    if i in train_images: # Check if the image belongs to training set
        train_img.append(i) # Add it to the list of train images
train_img = np.array(train_img)
face_train = face_train.loc[face_train['image'].isin(train_img)]
y = face_train['label'].values
y = np.array(y)

In [4]:
face_train.iloc[0]

image    629290de48014b7c9f0d38a540269627.png
label                                      21
Name: 300, dtype: object

In [5]:
#y_train = np.zeros((y.shape[0],1000))
#y_train[np.arange(y.shape[0]), y] = 1
X_train = []
y_train = []
for i in range(face_train.shape[0]):
    image_url = face_train.iloc[i]["image"]
    imagePath = os.path.join("train",image_url)
    image = load_img(imagePath, target_size=(img_rows, img_cols))
    image = img_to_array(image)
    image = np.expand_dims(image, 0)
    image = imagenet_utils.preprocess_input(image)
    X_train.append(image)
    a = i %3
    y_train.append(a)
    
X_train = np.vstack(X_train)


In [6]:
X_train.shape

(300, 224, 224, 3)

In [7]:
y = np.array(y_train)

In [8]:
A=np.unique(y_train)
len(A)

3

In [9]:
datagen = ImageDataGenerator(
            featurewise_center=True,
            featurewise_std_normalization=True,
            rotation_range=20,
            zoom_range=0.15,
            width_shift_range=10,
            height_shift_range=10,
            shear_range=0.15,
            horizontal_flip=True,
            fill_mode="nearest")
datagen.fit(X_train)
iterator_flow = datagen.flow(X_train, y, batch_size = 9)


In [10]:
dem = 0
X_au = []
y_au = []
for X_batch, y_batch in iterator_flow:
    X_batch = X_batch.reshape(-1, 224, 224, 3)
    y_au.extend(y_batch)
    X_au.extend(X_batch)
    dem = dem + 1
    if dem > 30:
        break


In [11]:
X_au = np.array(X_au)

In [12]:
X_au.shape

(279, 224, 224, 3)

In [13]:
y_au

[2,
 2,
 1,
 0,
 2,
 0,
 0,
 2,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 2,
 2,
 0,
 0,
 1,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 0,
 2,
 1,
 2,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 2,
 2,
 0,
 1,
 1,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 1,
 2,
 0,
 2,
 1,
 2,
 0,
 0,
 2,
 1,
 2,
 0,
 1,
 0,
 1,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 2,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 0,
 0,
 1,
 2,
 2,
 0,
 2,
 1,
 2,
 1,
 1,
 0,
 2,
 0,
 1,
 1,
 1,
 0,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 1,
 0,
 2,
 0,
 1,
 1,
 2,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 0,


In [14]:
y_au = np.array(y_au)
y_au.shape

(279,)

In [15]:
def convnet_model_():
    vgg_model = applications.VGG16(weights=None, include_top=False, input_shape=(224, 224, 3))
    x = vgg_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(4096, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = BatchNormalization(axis = 1, name = 'bn0')(x)
    convnet_model = Model(inputs=vgg_model.input, outputs=x)
    return convnet_model

def deep_rank_model():
    convnet_model = convnet_model_()

    first_input = Input(shape=(224, 224, 3))
    first_conv = Conv2D(96, kernel_size=(8,8), strides=(16,16), padding='same')(first_input)
    first_max = MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='same')(first_conv)
    first_max = Flatten()(first_max)
    first_max = BatchNormalization(axis = 1, name = 'bn1')(first_max)
    
    second_input = Input(shape=(224, 224, 3))
    second_conv = Conv2D(96, kernel_size=(8,8), strides=(32,32), padding='same')(second_input)
    second_max = MaxPooling2D(pool_size=(7,7), strides=(4,4), padding='same')(second_conv)
    second_max = Flatten()(second_max)
    second_max = BatchNormalization(axis = 1, name = 'bn2')(second_max)
                       
    merge_one = concatenate([first_max, second_max])
    merge_two = concatenate([merge_one, convnet_model.output])
    emb = Dense(4096)(merge_two)
    emb = Dense(128)(emb)
    #emb = BatchNormalization(axis = 1, name = 'bn3')(emb)
                        
    final_model = Model(inputs=[first_input, second_input, convnet_model.input], outputs=emb)

    return final_model

    

In [16]:
deep_rank_model = deep_rank_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [17]:
_EPSILON = K.epsilon()
def _loss_tensor(y_true, y_pred):
    y_pred = K.clip(y_pred, _EPSILON, 1.0 - _EPSILON)
    loss = 0.
    g = 1.
    for i in range(0, batch_size, 3):
        try:
            q_embedding = y_pred[i]
            p_embedding = y_pred[i+1]
            n_embedding = y_pred[i+2]
            D_q_p = K.sqrt(K.sum((q_embedding - p_embedding)**2))
            D_q_n = K.sqrt(K.sum((q_embedding - n_embedding)**2))
            loss = loss + g + D_q_p - D_q_n
        except:
            continue
    loss = loss/batch_size*3
    return K.maximum(loss, 0)

In [18]:
len(X_au)

279

In [19]:
def image_batch_generator(X_au, y_au, batch_size):
         
    y_au = np.array(y_au)
    while True:
        batch_paths = np.random.choice(a = len(X_au), size = batch_size//3)
        input_1 = []
        # print(2)
        for i in batch_paths:
            pos = np.where(y_au == y_au[i])[0]
            neg = np.where(y_au != y_au[i])[0]
            
            j = np.random.choice(pos)
            while j == i:
                j = np.random.choice(pos)
                # print(3)
            k = np.random.choice(neg)
            while k == i:
                k = np.random.choice(neg)
                # print(4)
            input_1.append(X_au[i])
            input_1.append(X_au[j])
            input_1.append(X_au[k])

        input_1 = np.array(input_1)
        input = [input_1, input_1, input_1]
        yield(input, np.zeros((batch_size, )))
        

In [20]:
deep_rank_model.compile(loss=_loss_tensor, optimizer=SGD(lr=0.001, momentum=0.9, nesterov=True))
deep_rank_model.fit_generator(generator=image_batch_generator(X_au, y_au, batch_size),steps_per_epoch=len(X_au)//batch_size,epochs=10)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
11/11 [==============================] - 226s 21s/step - loss: 0.9655
Epoch 2/10
11/11 [==============================] - 241s 22s/step - loss: 1.2127
Epoch 3/10
11/11 [==============================] - 212s 19s/step - loss: 1.1045
Epoch 4/10
11/11 [==============================] - 213s 19s/step - loss: 1.0866
Epoch 5/10
11/11 [==============================] - 210s 19s/step - loss: 0.6981
Epoch 6/10
11/11 [==============================] - 218s 20s/step - loss: 1.1992
Epoch 7/10
 2/11 [====>.........................] - ETA: 3:07 - loss: 0.5366

KeyboardInterrupt: 